# Tunier


In [ ]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

In [ ]:
%run ./nmm-alpha-beta-pruning.ipynb
%run ./nmm-minimax.ipynb

In [ ]:
from typing import Optional, List, Callable

## Spiel

In [ ]:
class Match():
    def __init__(
        self,
        white: ArtificialIntelligence, black: ArtificialIntelligence,
        start_state = s0, start_player = 'w',
        max_turns: int = 250, max_state_replayed: int = 5, max_states_without_mill: int = 30,
        name: str = ""
    ):
        self.white = white
        self.black = black
        
        self.state = start_state
        self.player = start_player
        self.max_turns = max_turns
        self.max_state_replayed = max_state_replayed
        self.max_states_without_mill = max_states_without_mill
        self.name = name
        
        self.log = [start_state]
        self.no_mill_played = 0

In [ ]:
def __repr__(self: Match):
    return f"Match(name='{self.name}', white={type(self.white).__name__}, " + \
           f"black={type(self.black).__name__}, max_turns={self.max_turns})"

Match.__repr__ = __repr__
del __repr__

In [ ]:
class MatchResult():
    def __init__(self, winner: str, log: List, reason: str):
        self.winner = winner
        self.log    = log
        self.reason = reason
    
    def __repr__(self):
        return f"MatchResult(winner='{self.winner}', log={len(self.log): >3}, reason='{self.reason}')"

In [ ]:
def current_ai(self: Match) -> ArtificialIntelligence:
    if self.player == 'w':
        return self.white
    return self.black

Match.current_ai = current_ai
del current_ai

In [ ]:
def check_remis(self: Match) -> Optional[MatchResult]:
    if len(self.log) >= self.max_turns:
        return MatchResult(
            winner = ' ',
            log    = self.log,
            reason = f"Reached max_turns after {self.max_turns} turns"
        )
    
    if self.log.count(self.state) >= self.max_state_replayed:
        return MatchResult(
            winner = ' ',
            log    = self.log,
            reason = f"State has been replayed for {self.log.count(self.state)} turns"
        )
    
    if self.no_mill_played >= self.max_states_without_mill:
        return MatchResult(
            winner = ' ',
            log    = self.log,
            reason = f"No mill has been played for {self.no_mill_played} turns"
        )
    return None

Match.check_remis = check_remis
del check_remis

In [ ]:
def play(self: Match) -> MatchResult:
    while True:
        if finished(self.state, self.player):
            return MatchResult(
                winner = ('w', ' ', 'b')[utility(self.state, 'b')+1],
                log    = self.log,
                reason = f"A player won the match"
            )
        
        mills_before = findMills(self.state[1], self.player)
        remis = self.check_remis()
        if remis is not None:
            return remis
        
        bestMoves = self.current_ai().bestMoves(self.state, self.player)
        self.state  = bestMoves.choice()
        self.player = opponent(self.player)
        
        self.log.append(self.state)
        if countNewMills(self.state[1], mills_before, self.player) <= 0:
            self.no_mill_played += 1
        else:
            self.no_mill_played = 0
        
        print(f"{self.name} t={len(self.log): <3} {self.player} found {len(bestMoves.states): >2} " + \
              f"states with value={bestMoves.value}")
    

Match.play = play
del play

## Runde

In [ ]:
from multiprocessing import cpu_count, Pool

In [ ]:
class Round():
    def __init__(
        self,
        white: ArtificialIntelligence, black: ArtificialIntelligence,
        instances: int
    ):
        self.white = white
        self.black = black
        self.instances = instances

In [ ]:
def __repr__(self: Round):
    return f"Round(white={type(self.white).__name__}, black={type(self.black).__name__}, instances={self.instances})"

Round.__repr__ = __repr__
del __repr__

In [ ]:
def execute(match):
    idx, rnd = match
    random.seed(idx)
    return rnd.play()

In [ ]:
def play(self: Round) -> List[MatchResult]:
    matches = [
        Match(
            self.white() if callable(self.white) else self.white,
            self.black() if callable(self.black) else self.black,
            name = f"r={i: <2}"
        )
        for i in range(self.instances)
    ]
    with Pool(self.instances) as pool:
        return pool.map(execute, enumerate(matches))

Round.play = play
del play

## Tunier

In [ ]:
class Tournament():
    def __init__(
        self,
        participants: List[ArtificialIntelligence],
        instances_per_round: int = cpu_count(),
        name: str = "unnamed"
    ):
        self.participants = participants
        self.instances_per_round = instances_per_round
        self.name = name

In [ ]:
def save(self: Tournament, idx: int, rnd: Round, results: List[MatchResult]):
    with open(f"round-{self.name}-{idx+1}.txt", "w") as file:
        file.write(f"Round: {idx+1}\n")
        
        file.write(f"\nPlayer:\n")
        white = rnd.white() if callable(rnd.white) else rnd.white
        black = rnd.black() if callable(rnd.black) else rnd.black
        file.write(f"  white: {white}\n")
        file.write(f"  black: {black}\n")
        
        file.write(f"\nResult:\n")
        file.write(f"  remis: {sum(result.winner==' ' for result in results)}\n")
        file.write(f"  white: {sum(result.winner=='w' for result in results)}\n")
        file.write(f"  black: {sum(result.winner=='b' for result in results)}\n")
        
        for midx, result in enumerate(results):
            file.write(f"\nMatch {midx+1}:\n")
            file.write(f"  Winner: '{result.winner}'\n")
            file.write(f"  Reason: {result.reason}\n")
            file.write(f"  Log:\n")
            for sidx, state in enumerate(result.log):
                file.write(f"    {sidx+1: >3}. {state}\n")

Tournament.save = save
del save

In [ ]:
def play(self: Tournament):
    rounds = [
        Round(a, b, self.instances_per_round)
        for a in self.participants
        for b in self.participants
        if a != b
    ]
    
    for idx, rnd in enumerate(rounds):
        results = rnd.play()
        print(f" === === === Round {idx+1: >2}/{len(rounds)} finished === === ===")
        self.save(idx, rnd, results)

Tournament.play = play
del play